In [1]:
import pandas as pd
import re
import os
import numpy as np

In [19]:
def leer_condicion(condition):
    # Para las columnas de texto, busca patrones del tipo 'variable = (vacío)' o 'variable no es (vacío)'
    text_var_pattern = r'(\w+)\s*(==|!=)\s*\((vacío|vacio)\)'
    text_var_matches = re.findall(text_var_pattern, condition)

    for var, op in text_var_matches:
        if op == '==':
            condition = condition.replace(f'{var} {op} (vacío)', f'{var} == ""')
            condition = condition.replace(f'{var} {op} (vacio)', f'{var} == ""')
        elif op == '!=':
            condition = condition.replace(f'{var} {op} (vacío)', f'{var} != ""')
            condition = condition.replace(f'{var} {op} (vacio)', f'{var} != ""')

    # Reemplaza los símbolos y frases con su equivalente en Python
    condition = condition.replace('<=', '<=')
    condition = condition.replace('=', '==').replace('<>', '!=').replace(">==", ">=").replace("<==","<=").replace("Y", "y")
    condition = condition.replace(' y ', ' & ').replace(' o ', ' | ').replace('NO ESTA EN', 'not in')

    # Para las demás columnas, asume que son numéricas y reemplaza 'no es (vacío)' por '!= np.nan' y 'es (vacío)' por '== np.nan'
    condition = condition.replace(' no es (vacío)', '!="NaN"')
    condition = condition.replace(' no es vacío', '!="NaN"')
    condition = condition.replace(' es (vacío)', '=="NaN"')
    condition = condition.replace(' es vacío', '=="NaN"')

    condition = condition.replace("NA", 'np.nan')

    # Reemplaza las comparaciones entre variables para que sean legibles en Python
    condition = re.sub(r'(\w+)\s*(<=|>=|<|>|==|!=)\s*(\w+)', r'\1 \2 \3', condition)

    # Si "está en" se encuentra en la condición, lo reemplaza por la sintaxis correcta en Python
    if "está en" in condition:
        condition = re.sub(r'(\w+)\s+está en\s+(\(.*?\))', r'\1 in \2', condition)
    
    # Agrega paréntesis alrededor de la condición
    condition = '(' + condition + ')'
    return condition


# Función para filtrar base de datos dada una query
def filter_base(df, conditions):
    #df = pd.read_spss(route)
    filter = leer_condicion(conditions)
    df_filtered = df.query(filter, local_dict={'np': np})
    return df_filtered


In [9]:
#leer_condicion("PPA03 < 7 y P09A01A es (vacío) | P09A01B es (vacío) |  P09A01C es (vacío) | P09A02A es (vacío) | P09A02B es (vacío) | P09A02C es vacío | P09A03A es vacío | P09A03B es (vacío) |  P09A03C es (vacío) | P09A04A es (vacío) | P09A04B es (vacío) | P09A04C es (vacío) | P09A05A es (vacío) | P09A05B es (vacío) | P09A05C es (vacío)")
filter_base("BasePrueba.sav", "P06B16A = 2 & P06B16B no es (vacío)")

,P09H01A,P09H01B,P09H01C,P09H01D,P07A02B,P07A06C,P06B06B,P06B25B,P10E10B,PPA01A,...,PPB01,PPB04,PPC03,PPD04,PPD07,PPD08,PPC01,PPC02B,PPC04,PPD06


In [186]:
# Función para devolver inconsistencias dado un analista, capitulo, seccion en especifico
def process_specific_data(capitulo, seccion, analista, route_expressions, route_base):
    try:
        # Abrir archivo de expresiones para filtrar
        expressions = pd.read_excel(route_expressions)
        
        # Crear lista con expresiones para filtrar
        expressions = list(expressions[(expressions["Analista"] == analista) & (expressions["Capítulo"] == capitulo) & (expressions["Sección"] == seccion)]["Condición o Criterio"])
        
        # Crear archivo tipo ExcelWriter para exportar en diferentes pestañas
        writer = pd.ExcelWriter("C{}S{}.xlsx".format(capitulo,seccion))
        
        # Leer filtros y tomar subconjuntos de la base
        for i in range(len(expressions)):
            try:
                Validacion = filter_base(route_base, expressions[i])  # Aplicar filtro a la base de datos
                sheet_name = "S{}V{}".format(capitulo, seccion, i)  # Generar el nombre de la hoja
                Validacion.to_excel(writer, sheet_name=sheet_name)  # Exportar subconjunto de datos a una hoja de Excel
            except Exception as e:
                print(f"Error al procesar la expresión {expressions[i]}: {e}")  # Manejar error específico de una expresión

        writer.save()  # Guardar el archivo de Excel con las hojas generadas
        print("Proceso completado exitosamente.")  # Indicar que el proceso ha finalizado con éxito
    
    except Exception as e:
        print(f"Error general: {e}")  # Manejar error general en caso de problemas durante el proceso



Error al procesar la expresión P01A01 = 1 P01A05A = 2 & P01A05B = 2 & P01A05B = 2 & P01A05C = 2 & P01A05D = 2 & P01A05E = 2 & P01A05F = 2: Python keyword not valid identifier in numexpr query (<unknown>, line 1)
Proceso completado exitosamente.


C:\Users\mchinchilla\AppData\Local\Temp\ipykernel_20308\4022916948.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()  # Guardar el archivo de Excel con las hojas generadas


In [15]:
# Funcion de lectura de expresiones generales
def process_general_data(route_expressions, route_base):
    df = pd.read_spss(route_base)
    try:
        # Abrir archivo de expresiones para filtrar
        expressions = pd.read_excel(route_expressions)
        grouped = expressions.groupby(["Capítulo", "Sección"])

        # Crear lista con expresiones para filtrar
        tuplas_chap_sec = [(name[0], name[1]) for name, _ in grouped]

        # Leer filtros y tomar subconjuntos de la base
        for capitulo, seccion in tuplas_chap_sec:
            # Crear carpeta por capitulo
            folder_name = "C{}".format(capitulo)
            ruta_carpeta = os.path.join(os.getcwd(), folder_name)
            if not os.path.exists(ruta_carpeta):
                os.mkdir(ruta_carpeta)
            conditions = list(expressions[(expressions["Capítulo"] == capitulo) & (expressions["Sección"] == seccion)]["Condición o Criterio"])
            for condition in conditions:
                try:
                    Validacion = filter_base(df, condition)  # Aplicar filtro a la base de datos
                    sheet_name = "S{}V{}".format(seccion, conditions.index(condition))  # Generar el nombre de la hoja
                    filename = os.path.join(ruta_carpeta, "S{}.xlsx".format(seccion))  # Crea la ruta completa al archivo
                    Validacion.to_excel(filename, sheet_name=sheet_name)  # Exportar subconjunto de datos a una hoja de Excel
                except Exception as e:
                    print(f"Error al procesar la expresión {condition}: {e}")  # Manejar error específico de una expresión

    except Exception as e:
        print(f"Error general: {e}")  # Manejar error general en caso de problemas durante el proceso

In [23]:
process_general_data("Expresiones.xlsx", "BasePrueba.sav")

Error al procesar la expresión P01A01 = 5 y P01A04 no está en (4,7,98): Python keyword not valid identifier in numexpr query (<unknown>, line 1)
Error al procesar la expresión P01C01 = 2 y P01C02A = 1 y P01C02B = 1 y P01C02C = 1 y P01C02D = 1 y P01C02E = 1 y P01C02F = 1 y P01C02G = 1 y P01C02H = 1 y P01C02I = 1 y P01C02J = 1 y P01C02K = 1 y P01C02L = 1 y P01C02M = 1: name 'P01C02E' is not defined
Error al procesar la expresión P01G01 NO ESTA EN (1,2,3,4): name 'P01G01' is not defined
Error al procesar la expresión P01H09 = 1 y (P01H10 = (VACIO)
P01H11 = (VACIO)
P01H12 = (VACIO)
P01H13 = (VACIO)
P01H14 = (VACIO)
P01H15 = (VACIO)
P01H16 = (VACIO) ): multi-line expressions are only valid in the context of data, use DataFrame.eval
Error al procesar la expresión P01H09 = 2 y (P01H10 <> (VACIO)
P01H11 <> (VACIO)
P01H12 <> (VACIO)
P01H13 <> (VACIO)
P01H14 <> (VACIO)
P01H15 <> (VACIO)
P01H16 <> (VACIO) ): multi-line expressions are only valid in the context of data, use DataFrame.eval


Exception ignored in: <function ZipFile.__del__ at 0x00000286C2EFE8C0>
Traceback (most recent call last):
  File "c:\Users\Mariajose\AppData\Local\Programs\Python\Python310\lib\zipfile.py", line 1819, in __del__
    self.close()
  File "c:\Users\Mariajose\AppData\Local\Programs\Python\Python310\lib\zipfile.py", line 1836, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


Error al procesar la expresión P04A02 >= 1 o P04A02 <= 12 y P04A01A = 1: Unordered Categoricals can only compare equality or not
Error al procesar la expresión P04A03 >= 1 o P04A03 <= 28 y P04A01A = 1 o P04A03 = 97 y P04A01A = 1 o P04A03 = 99 y P04A01A = 1: Unordered Categoricals can only compare equality or not
Error al procesar la expresión P04A05 >= 1 o P04A05 <= 12 y P04A04A = 1: Unordered Categoricals can only compare equality or not
Error al procesar la expresión P04A06 >= 1 o P04A06 <= 28 y P04A04A = 1 o P04A06 = 97 y P04A04A = 1 o P04A06 = 99 y P04A04A = 1: Unordered Categoricals can only compare equality or not
Error al procesar la expresión P04A07 > 1 y P04A08 no es (vacío): Unordered Categoricals can only compare equality or not
Error al procesar la expresión PPA03 < P05A02 y P05A02 <> 97 y PPA03 < P05A02 y P05A02 <> 96 o PPA03 < P05A03 y P05A03 <> 97 o PPA03 < P05A05 y P05A05 <> 97 o PPA03 < P05A06 y P05A06 <> 97 : Unordered Categoricals can only compare equality or not
Err

KeyboardInterrupt: 